<a href="https://colab.research.google.com/github/Veranzi/AfyaSasa/blob/main/Afyasasa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install modelbit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [1]:
# install modelbit
!pip install modelbit

# run on top of your notebook
import modelbit
mb = modelbit.login()

In [2]:
import pandas as pd
from io import StringIO
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
# --- Data Loading ---
# Load your Excel file
df = pd.read_excel("/content/drive/MyDrive/hackathon/Ovarian Cyst Track Data.xlsx")

# Preview
df.head()

,Patient ID,Age,Menopause Status,Cyst Size cm,Cyst Growth Rate cm/month,CA 125 Level,Ultrasound Features,Reported Symptoms,Recommended Management,Date of Exam,Region
0,OC-1000,52,Post-menopausal,3.2,-0.34,19,Septated cyst,"Pelvic pain, Nausea, Bloating",Observation,2025-05-23,Eldoret
1,OC-1001,62,Post-menopausal,7.9,0.84,111,Septated cyst,Bloating,Medication,2025-04-27,Loitoktok
2,OC-1002,59,Post-menopausal,2.2,0.50,123,Hemorrhagic cyst,"Pelvic pain, Irregular periods, Bloating",Referral,2025-01-05,Eldoret
3,OC-1003,64,Post-menopausal,5.5,1.11,116,Solid mass,"Nausea, Irregular periods",Medication,2025-03-11,Loitoktok
4,OC-1004,43,Post-menopausal,4.0,0.90,98,Hemorrhagic cyst,"Irregular periods, Bloating, Nausea",Surgery,2025-02-17,Eldoret


In [49]:
# Take a quick look
print(df.shape)
print(df.dtypes)
df.head()

(100, 11)
Patient ID                           object
Age                                   int64
Menopause Status                     object
Cyst Size cm                        float64
Cyst Growth Rate cm/month           float64
CA 125 Level                          int64
Ultrasound Features                  object
Reported Symptoms                    object
Recommended Management               object
Date of Exam                 datetime64[ns]
Region                               object
dtype: object


,Patient ID,Age,Menopause Status,Cyst Size cm,Cyst Growth Rate cm/month,CA 125 Level,Ultrasound Features,Reported Symptoms,Recommended Management,Date of Exam,Region
0,OC-1000,52,Post-menopausal,3.2,-0.34,19,Septated cyst,"Pelvic pain, Nausea, Bloating",Observation,2025-05-23,Eldoret
1,OC-1001,62,Post-menopausal,7.9,0.84,111,Septated cyst,Bloating,Medication,2025-04-27,Loitoktok
2,OC-1002,59,Post-menopausal,2.2,0.50,123,Hemorrhagic cyst,"Pelvic pain, Irregular periods, Bloating",Referral,2025-01-05,Eldoret
3,OC-1003,64,Post-menopausal,5.5,1.11,116,Solid mass,"Nausea, Irregular periods",Medication,2025-03-11,Loitoktok
4,OC-1004,43,Post-menopausal,4.0,0.90,98,Hemorrhagic cyst,"Irregular periods, Bloating, Nausea",Surgery,2025-02-17,Eldoret


Drop non‑predictive columns

In [50]:
df = df.drop(columns=[
    "Patient ID",       # Identifier
    "Date of Exam",     # Date metadata
    "Region"            # Geographic metadata
])

In [ ]:

# Drop rows with missing values
df.dropna(inplace=True)

Unique values

In [51]:
print(df["Menopause Status"].unique())
print(df["Ultrasound Features"].unique())
print(df["Reported Symptoms"].unique())


['Post-menopausal' 'Pre-menopausal']
['Septated cyst' 'Hemorrhagic cyst' 'Solid mass' 'Complex cyst'
 'Simple cyst']
['Pelvic pain, Nausea, Bloating' 'Bloating'
 'Pelvic pain, Irregular periods, Bloating' 'Nausea, Irregular periods'
 'Irregular periods, Bloating, Nausea'
 'Fatigue, Bloating, Irregular periods'
 'Bloating, Pelvic pain, Irregular periods' 'Irregular periods'
 'Fatigue, Nausea, Pelvic pain' 'Bloating, Irregular periods'
 'Fatigue, Pelvic pain, Nausea' 'Pelvic pain, Bloating, Fatigue'
 'Bloating, Nausea, Irregular periods'
 'Irregular periods, Fatigue, Nausea' 'Nausea, Fatigue, Pelvic pain'
 'Irregular periods, Fatigue, Bloating'
 'Nausea, Irregular periods, Bloating' 'Bloating, Nausea'
 'Bloating, Fatigue' 'Pelvic pain, Irregular periods'
 'Irregular periods, Nausea' 'Bloating, Nausea, Fatigue'
 'Pelvic pain, Nausea' 'Irregular periods, Fatigue' 'Nausea'
 'Bloating, Pelvic pain' 'Pelvic pain' 'Fatigue, Nausea, Bloating'
 'Fatigue' 'Nausea, Bloating, Irregular periods' 'Fa

In [53]:
# Extract target
y = df["Recommended Management"]
X = df.drop(columns=["Recommended Management"])

In [54]:
# --- Preprocessing ---
# Split multi-symptom into dummy variables
symptom_dummies = X["Reported Symptoms"].str.get_dummies(sep=", ")
X = pd.concat([X.drop(columns=["Reported Symptoms"]), symptom_dummies], axis=1)

In [55]:
# Label Encoding for categorical
X["Menopause Status"] = X["Menopause Status"].map({
    'Post-menopausal': 0,
    'Pre-menopausal': 1
})
X["Ultrasound Features"] = X["Ultrasound Features"].map({
    'Septated cyst': 0,
    'Hemorrhagic cyst': 1,
    'Solid mass': 2,
    'Complex cyst': 3,
    'Simple cyst': 4
})

In [56]:
# Encode target
target_le = LabelEncoder()
y_encoded = target_le.fit_transform(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)


In [57]:
# Feature scaling
numerical = ["Age", "Cyst Size cm", "Cyst Growth Rate cm/month", "CA 125 Level"]
scaler = StandardScaler()
X_train[numerical] = scaler.fit_transform(X_train[numerical])
X_test[numerical] = scaler.transform(X_test[numerical])

Fit a classifier
Here we use a Random Forest, but you can swap in XGBoost, LogisticRegression, etc.

In [58]:
import pandas as pd
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# Step 9a: Define the classifiers to compare
classifiers = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Support Vector Machine': SVC(kernel='rbf', probability=True, random_state=42)
}

# Step 9b: Set up stratified 5‑fold CV
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Step 9c: Evaluate each classifier
results = []
for name, clf in classifiers.items():
    scores = cross_val_score(clf, X, y, cv=cv, scoring='accuracy')
    results.append({
        'Classifier': name,
        'Mean CV Accuracy': scores.mean(),
        'Std CV Accuracy': scores.std()
    })

# Step 9d: Create and display a DataFrame of the results
results_df = pd.DataFrame(results).sort_values('Mean CV Accuracy', ascending=False)
print(results_df.to_string(index=False))


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

            Classifier  Mean CV Accuracy  Std CV Accuracy
     Gradient Boosting              0.31         0.124097
Support Vector Machine              0.27         0.040000
   K-Nearest Neighbors              0.25         0.054772
         Random Forest              0.22         0.050990
   Logistic Regression              0.21         0.058310


Hyperparameter Tuning for Gradient Boosting

In [59]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold

# Define parameter grid
param_dist = {
    'n_estimators': [50, 100, 200, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6],
    'subsample': [0.6, 0.8, 1.0],
    'min_samples_leaf': [1, 2, 4]
}

gb = GradientBoostingClassifier(random_state=42)

rs = RandomizedSearchCV(
    estimator=gb,
    param_distributions=param_dist,
    n_iter=20,                # number of random combinations
    cv=StratifiedKFold(5),    # 5‑fold CV
    scoring='accuracy',
    random_state=42,
    n_jobs=-1
)

rs.fit(X_train, y_train)

print("Best GB Params:", rs.best_params_)
print("Best CV Score:  ", rs.best_score_)


Best GB Params: {'subsample': 0.8, 'n_estimators': 500, 'min_samples_leaf': 2, 'max_depth': 4, 'learning_rate': 0.1}
Best CV Score:   0.3375


Retrain on Full Training Set

In [60]:
best_gb = rs.best_estimator_
best_gb.fit(X_train, y_train)


GradientBoostingClassifier(max_depth=4, min_samples_leaf=2, n_estimators=500,
                           random_state=42, subsample=0.8)

Evaluate on Test Set

In [61]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = best_gb.predict(X_test)

print("Test Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=target_le.classes_))

cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)


Test Accuracy: 0.1
              precision    recall  f1-score   support

  Medication       0.33      0.33      0.33         6
 Observation       0.00      0.00      0.00         5
    Referral       0.00      0.00      0.00         4
     Surgery       0.00      0.00      0.00         5

    accuracy                           0.10        20
   macro avg       0.08      0.08      0.08        20
weighted avg       0.10      0.10      0.10        20

Confusion Matrix:
 [[2 2 1 1]
 [0 0 2 3]
 [3 1 0 0]
 [1 3 1 0]]


Feature Importances

In [62]:
import pandas as pd

feat_imp = pd.Series(
    best_gb.feature_importances_,
    index=X.columns
).sort_values(ascending=False)

print("Top 10 Features:\n", feat_imp.head(10))


Top 10 Features:
 Cyst Growth Rate cm/month    0.243614
Age                          0.178040
CA 125 Level                 0.175720
Cyst Size cm                 0.175372
Ultrasound Features          0.078707
Fatigue                      0.035660
Nausea                       0.028680
Irregular periods            0.027138
Pelvic pain                  0.023272
Bloating                     0.018861
dtype: float64


In [63]:
# Train model
best_gb = GradientBoostingClassifier()
best_gb.fit(X_train, y_train)


GradientBoostingClassifier()

In [64]:
# Feature names for inference
final_features = X.columns

Creating an Inference function

In [65]:
# ---------------------------------------
# Inference Function
# ---------------------------------------

def predict_ovarian_cyst_management(
    Age: int,
    Menopause_Status: str,
    Cyst_Size_cm: float,
    Cyst_Growth_Rate_cm_per_month: float,
    CA125_Level: float,
    Ultrasound_Features: str,
    Reported_Symptoms: str
) -> dict:
    # Build input DataFrame
    input_df = pd.DataFrame([{
        "Age": Age,
        "Menopause Status": 0 if Menopause_Status == "Post-menopausal" else 1,
        "Cyst Size cm": Cyst_Size_cm,
        "Cyst Growth Rate cm/month": Cyst_Growth_Rate_cm_per_month,
        "CA 125 Level": CA125_Level,
        "Ultrasound Features": {
            'Septated cyst': 0,
            'Hemorrhagic cyst': 1,
            'Solid mass': 2,
            'Complex cyst': 3,
            'Simple cyst': 4
        }.get(Ultrasound_Features, -1)
    }])

    # Process symptoms
    symptom_dummies_input = pd.Series(Reported_Symptoms.split(", ")).str.strip().tolist()
    for col in final_features:
        if col not in input_df.columns:
            input_df[col] = 1 if col in symptom_dummies_input else 0

    # Reorder columns to match training
    input_df = input_df[final_features]

    # Scale numericals
    input_df[numerical] = scaler.transform(input_df[numerical])

    # Predict
    probs = best_gb.predict_proba(input_df)[0]
    classes = target_le.inverse_transform(np.arange(len(probs)))
    return dict(zip(classes, probs))

In [66]:
# ---------------------------------------
# Example: Call function
# ---------------------------------------

result = predict_ovarian_cyst_management(
    Age=59,
    Menopause_Status="Post-menopausal",
    Cyst_Size_cm=2.2,
    Cyst_Growth_Rate_cm_per_month=0.50,
    CA125_Level=123,
    Ultrasound_Features="Hemorrhagic cyst",
    Reported_Symptoms="Pelvic pain, Irregular periods, Bloating"
)

print("Predicted Management Probabilities:\n", result)

Predicted Management Probabilities:
 {'Medication': 0.001261544091056755, 'Observation': 0.0005343283812438806, 'Referral': 0.9911108017013976, 'Surgery': 0.007093325826301756}


Now to deploy this pipeline

In [67]:
# deploy predict_ovarian_cyst_management
mb.deploy(predict_ovarian_cyst_management)

In [71]:
!curl -s -XPOST "https://veranziverah.app.modelbit.com/v1/predict_ovarian_cyst_management/latest" -d '{"data":[59,"Post-menopausal",2.2,0.50,123,"Hemorrhagic cyst","Pelvic pain, Irregular periods, Bloating"]}' | json_pp


{
   "data" : {
      "Medication" : 0.00126154409105676,
      "Observation" : 0.000534328381243881,
      "Referral" : 0.991110801701398,
      "Surgery" : 0.00709332582630175
   }
}
